In [16]:
import os
import sys
import numpy as np
import pandas as pd
import librosa
import librosa.effects
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# --- Step 1: Configuration & Parameters ---
LABEL_PATH_1_2 = "/home/punith/Desktop/cHEAL 2.o/Labels.xlsx"
AUDIO_DIR_1_2 = "/home/punith/Desktop/cHEAL 2.o/RespiratoryDatabase@TR"
MODEL_SAVE_PATH_1_2 = "best_copd_1_2_model.keras"

N_MELS, MAX_LEN, EPOCHS, BATCH_SIZE = 128, 150, 25, 32
NOISE_FACTOR, TIME_SHIFT_MAX_SEC, PITCH_SHIFT_STEPS, TIME_STRETCH_RATE = 0.005, 0.2, 4, 0.8
INITIAL_LEARNING_RATE = 0.001


# --- Step 2: Helper Functions ---
def add_gaussian_noise(y, noise_factor=NOISE_FACTOR):
    return y + noise_factor * np.random.randn(len(y))

def time_shift(y, sr, shift_max_sec=TIME_SHIFT_MAX_SEC):
    return np.roll(y, int(sr*np.random.uniform(-shift_max_sec, shift_max_sec)))

def extract_log_mel_spectrogram(y, sr):
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    log_mel = librosa.power_to_db(mel_spec)
    if log_mel.shape[1] < MAX_LEN:
        log_mel = np.pad(log_mel, ((0, 0), (0, MAX_LEN - log_mel.shape[1])), mode='constant')
    else:
        log_mel = log_mel[:, :MAX_LEN]
    return log_mel


# --- Step 3: Load Data with Final Validation ---
print("--- Step 3: Loading and Validating Data for COPD 1 vs 2 ---")
df = pd.read_excel(LABEL_PATH_1_2)
df_copd_1_2 = df[df["Diagnosis"].isin(["COPD1", "COPD2"])].copy()

if df_copd_1_2['Diagnosis'].nunique() < 2:
    raise ValueError("The Excel file must contain patients from both 'COPD1' and 'COPD2' diagnoses.")

df_copd_1_2['label_encoded'] = df_copd_1_2['Diagnosis'].apply(lambda x: 0 if x == 'COPD1' else 1)
label_dict_1_2 = dict(zip(df_copd_1_2["Patient ID"], df_copd_1_2["label_encoded"]))
patient_ids_1_2 = list(label_dict_1_2.keys())
patient_labels_1_2 = list(label_dict_1_2.values())

print("\nPerforming stratified patient-aware split...")
try:
    train_pids, test_pids, _, _ = train_test_split(
        patient_ids_1_2, patient_labels_1_2,
        test_size=0.25, random_state=42, stratify=patient_labels_1_2
    )
except ValueError as e:
    print(f"\nFATAL ERROR during train/test split: {e}")
    print("This usually means one class has only 1 patient, making stratification impossible.")
    raise

print(f"Total Patients: {len(patient_ids_1_2)}, Training PIDs: {len(train_pids)}, Testing PIDs: {len(test_pids)}")

X_train, y_train, X_test, y_test = [], [], [], []
for pid in patient_ids_1_2:
    label = label_dict_1_2[pid]
    is_training_patient = pid in train_pids
    for side in ['L', 'R']:
        for i in range(1, 7):
            fname = f"{pid}_{side}{i}.wav"
            fpath = os.path.join(AUDIO_DIR_1_2, fname)
            if not os.path.exists(fpath): continue
            try:
                y_audio, sr = librosa.load(fpath, sr=None)
                spec = extract_log_mel_spectrogram(y_audio, sr)
                if is_training_patient:
                    y_train.extend([label] * 5)
                    X_train.extend([
                        spec, extract_log_mel_spectrogram(add_gaussian_noise(y_audio, sr), sr),
                        extract_log_mel_spectrogram(time_shift(y_audio, sr), sr),
                        extract_log_mel_spectrogram(librosa.effects.pitch_shift(y=y_audio, sr=sr, n_steps=4), sr),
                        extract_log_mel_spectrogram(librosa.effects.time_stretch(y=y_audio, rate=1/TIME_STRETCH_RATE), sr)
                    ])
                elif pid in test_pids:
                    X_test.append(spec)
                    y_test.append(label)
            except Exception as e:
                print(f"Warning: Error processing {fname}: {e}")

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

# --- FINAL VALIDATION BLOCK (The definitive fix) ---
if len(np.unique(y_train)) < 2:
    raise ValueError(
        "\n\nFATAL ERROR: The training set ('y_train') ended up with only one class.\n"
        "This happens because audio files for the other class are MISSING from the directory.\n\n"
        f"Training was attempted with these Patient IDs: {train_pids}\n\n"
        "ACTION REQUIRED: Please verify that the audio files for ALL of these patients actually exist in this folder:\n"
        f"-> {AUDIO_DIR_1_2}\n"
    )

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
print("\n--- Final Data Shapes ---\n", f"X_train: {X_train.shape}, y_train: {y_train.shape}\n", f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# --- Step 4: Calculate Class Weights for 1 vs 2 ---
print("\n--- Step 4: Handling Class Imbalance for 1 vs 2 ---")
class_weights_array = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_1_2 = dict(enumerate(class_weights_array))
print(f"Counts: COPD1(0)={np.sum(y_train == 0)}, COPD2(1)={np.sum(y_train == 1)}")
print(f"Calculated Weights: {class_weights_1_2}")


# --- Step 5: Build the CNN Model ---
print("\n--- Step 5: Building the CNN Model for 1-2 Progression ---")
model_1_2 = Sequential([
    tf.keras.Input(shape=(N_MELS, MAX_LEN, 1)),
    Conv2D(32, (3, 3), activation='relu', padding='same'), BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu', padding='same'), BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu', padding='same'), BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
    Flatten(), Dense(128, activation='relu'), Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model_1_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model_1_2.summary()


--- Step 3: Loading and Validating Data for COPD 1 vs 2 ---

Performing stratified patient-aware split...
Total Patients: 12, Training PIDs: 9, Testing PIDs: 3

--- Final Data Shapes ---
 X_train: (540, 128, 150, 1), y_train: (540,)
 X_test: (36, 128, 150, 1), y_test: (36,)

--- Step 4: Handling Class Imbalance for 1 vs 2 ---
Counts: COPD1(0)=240, COPD2(1)=300
Calculated Weights: {0: np.float64(1.125), 1: np.float64(0.9)}

--- Step 5: Building the CNN Model for 1-2 Progression ---


2025-06-30 16:03:25.904082: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 150, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 150, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 75, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,718,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,812,417 (18.36 MB)

 Trainable params: 4,811,969 (18.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [17]:
# --- Step 6: Train the 1-2 Progression Model ---
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH_1_2, save_best_only=True, monitor='val_accuracy', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
print(f"\n--- Step 6: Starting Model Training for COPD 1-2 ---\n")
history_1_2 = model_1_2.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS, batch_size=BATCH_SIZE,
    callbacks=[early_stop, model_checkpoint, reduce_lr],
    class_weight=class_weights_1_2
)


--- Step 6: Starting Model Training for COPD 1-2 ---

Epoch 1/25


2025-06-30 16:03:39.613513: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78643200 exceeds 10% of free system memory.
2025-06-30 16:03:39.664811: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78643200 exceeds 10% of free system memory.
2025-06-30 16:03:39.706835: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78643200 exceeds 10% of free system memory.
2025-06-30 16:03:40.371572: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78643200 exceeds 10% of free system memory.
2025-06-30 16:03:40.371627: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 78643200 exceeds 10% of free system memory.


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 778ms/step - accuracy: 0.5472 - loss: 10.4778
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to best_copd_1_2_model.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 18s 813ms/step - accuracy: 0.5463 - loss: 10.3755 - val_accuracy: 0.3333 - val_loss: 5.3143 - learning_rate: 0.0010
Epoch 2/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 739ms/step - accuracy: 0.4847 - loss: 3.8411
Epoch 2: val_accuracy improved from 0.33333 to 0.66667, saving model to best_copd_1_2_model.keras
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 766ms/step - accuracy: 0.4870 - loss: 3.8005 - val_accuracy: 0.6667 - val_loss: 1.0821 - learning_rate: 0.0010
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 742ms/step - accuracy: 0.4820 - loss: 1.7950
Epoch 3: val_accuracy did not improve from 0.66667
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 754ms/step - accuracy: 0.4826 - loss: 1.7864 - val_accuracy: 0.6667 - val_loss: 0.8145 - learning_rate: 0.0010
Epoch 4/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.4926 - loss: 0.

In [18]:
# --- Step 7: Evaluate the Final 1-2 Model ---
print("\n--- Step 7: Evaluating Best Saved 1-2 Model ---")
model_1_2.load_weights(MODEL_SAVE_PATH_1_2)
loss, accuracy = model_1_2.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy (1 vs 2): {accuracy*100:.2f}%")
print(f"Final Test Loss (1 vs 2): {loss:.4f}")


--- Step 7: Evaluating Best Saved 1-2 Model ---

Final Test Accuracy (1 vs 2): 66.67%
Final Test Loss (1 vs 2): 1.0821


In [20]:
# --- Step 8: Patient-Level Analysis (Averaging Method) ---
def run_patient_analysis_by_average(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    print(f"\n--- 📈 Starting Analysis for '{target_diagnosis}' Patients (Averaging Scores) ---")
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    if not patient_ids: return None
    print(f"Found {len(patient_ids)} patients. Analyzing files...")
    results_list = []
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fpath = os.path.join(audio_dir, f"{pid}_{side}{i}.wav")
                if not os.path.exists(fpath): continue
                try:
                    y, sr = librosa.load(fpath, sr=None)
                    spec = extract_log_mel_spectrogram(y, sr)
                    prob = trained_model.predict(np.expand_dims(spec, axis=(0, -1)), verbose=0)[0][0]
                    scores.append(prob)
                except Exception as e:
                    print(f"Warning: Could not process file {fpath}: {e}")
        if scores:
            results_list.append({'Patient ID': pid, 'Avg_Score': np.mean(scores), 'Audio_Files_Found': len(scores)})
    results_df = pd.DataFrame(results_list).sort_values(by='Avg_Score', ascending=sort_ascending)
    return results_df.reset_index(drop=True)

def assess_copd2_patient(row):
    return "✅ Model Confident (Correctly resembles Stage 2)" if row['Avg_Score'] >= 0.5 else "⚠️ Model Lacks Confidence (False Negative)"

def assess_copd1_progression_risk(row):
    return "⚠️ High Risk (Resembles Stage 2 - False Positive)" if row['Avg_Score'] >= 0.5 else "✅ Low Risk (Correctly identified as Stage 1)"

# --- Analyze COPD2 Patients ---
copd2_avg_df = run_patient_analysis_by_average(model_1_2, LABEL_PATH_1_2, AUDIO_DIR_1_2, 'COPD2', True)
if copd2_avg_df is not None:
    copd2_avg_df['Assessment'] = copd2_avg_df.apply(assess_copd2_patient, axis=1)
    print("\n--- ✅ COPD2 Patient Confidence Results (Averaging Method) ---")
    print("Shows model's confidence in identifying patients known to have Stage 2. Avg_Score >= 0.5 is correct.")
    print(copd2_avg_df.to_string())

# --- Analyze COPD1 Patients ---
copd1_avg_df = run_patient_analysis_by_average(model_1_2, LABEL_PATH_1_2, AUDIO_DIR_1_2, 'COPD1', False)
if copd1_avg_df is not None:
    copd1_avg_df['Assessment'] = copd1_avg_df.apply(assess_copd1_progression_risk, axis=1)
    print("\n--- ⚠️ COPD1 Patient Progression Risk Results (Averaging Method) ---")
    print("Shows which Stage 1 patients are flagged as being at high risk of progressing to Stage 2.")
    print(copd1_avg_df.to_string())


--- 📈 Starting Analysis for 'COPD2' Patients (Averaging Scores) ---
Found 7 patients. Analyzing files...

--- ✅ COPD2 Patient Confidence Results (Averaging Method) ---
Shows model's confidence in identifying patients known to have Stage 2. Avg_Score >= 0.5 is correct.
  Patient ID  Avg_Score  Audio_Files_Found                                       Assessment
0       H042   0.923945                 12  ✅ Model Confident (Correctly resembles Stage 2)
1       H018   0.927974                 12  ✅ Model Confident (Correctly resembles Stage 2)
2       H044   0.942048                 12  ✅ Model Confident (Correctly resembles Stage 2)
3       H031   0.942467                 12  ✅ Model Confident (Correctly resembles Stage 2)
4       H038   0.950762                 12  ✅ Model Confident (Correctly resembles Stage 2)
5       H030   0.957373                 12  ✅ Model Confident (Correctly resembles Stage 2)
6       H028   0.966230                 12  ✅ Model Confident (Correctly resembles Sta

After data balancing

COPD 1 - 5 samples augmented to 6 types

COPD 2 - 7 samples augmented to 5 types

In [39]:
import os
import sys
import numpy as np
import pandas as pd
import librosa
import librosa.effects
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# --- Step 1: Configuration & Parameters ---
LABEL_PATH_1_2 = "/home/punith/Desktop/cHEAL 2.o/Labels.xlsx"
AUDIO_DIR_1_2 = "/home/punith/Desktop/cHEAL 2.o/RespiratoryDatabase@TR"
MODEL_SAVE_PATH_1_2 = "best_copd_1_2_oversampled_model.keras"

N_MELS, MAX_LEN, EPOCHS, BATCH_SIZE = 128, 150, 25, 32
NOISE_FACTOR, TIME_SHIFT_MAX_SEC, PITCH_SHIFT_STEPS, TIME_STRETCH_RATE = 0.005, 0.2, 4, 0.8
INITIAL_LEARNING_RATE = 0.001


# --- Step 2: Helper Functions ---
def add_gaussian_noise(y, noise_factor=NOISE_FACTOR): return y + noise_factor * np.random.randn(len(y))
def time_shift(y, sr, shift_max_sec=TIME_SHIFT_MAX_SEC): return np.roll(y, int(sr*np.random.uniform(-shift_max_sec, shift_max_sec)))
def extract_log_mel_spectrogram(y, sr):
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    log_mel = librosa.power_to_db(mel_spec)
    if log_mel.shape[1] < MAX_LEN: log_mel = np.pad(log_mel, ((0, 0), (0, MAX_LEN - log_mel.shape[1])), mode='constant')
    else: log_mel = log_mel[:, :MAX_LEN]
    return log_mel


# --- Step 3: Load Data and Apply Oversampling to Balance the Dataset ---
print("--- Step 3: Loading Data for COPD 1 vs 2 ---")
df = pd.read_excel(LABEL_PATH_1_2)
df_copd_1_2 = df[df["Diagnosis"].isin(["COPD1", "COPD2"])].copy()

if df_copd_1_2['Diagnosis'].nunique() < 2:
    raise ValueError("The Excel file must contain patients from both 'COPD1' and 'COPD2'.")

df_copd_1_2['label_encoded'] = df_copd_1_2['Diagnosis'].apply(lambda x: 0 if x == 'COPD1' else 1) # COPD1=0, COPD2=1
label_dict_1_2 = dict(zip(df_copd_1_2["Patient ID"], df_copd_1_2["label_encoded"]))
patient_ids_1_2 = list(label_dict_1_2.keys())
patient_labels_1_2 = list(label_dict_1_2.values())

print("\nPerforming stratified patient-aware split...")
try:
    train_pids, test_pids, y_train_pids_labels, _ = train_test_split(
        patient_ids_1_2, patient_labels_1_2,
        test_size=0.25, random_state=42, stratify=patient_labels_1_2
    )
except ValueError as e:
    raise ValueError(f"\nFATAL ERROR: {e}\nThis means a class has only 1 patient.") from e

# --- OVERSAMPLING LOGIC START ---
print("\nCalculating oversampling rate to balance the training set...")
base_augmentations = 5
num_copd1_train = y_train_pids_labels.count(0)
num_copd2_train = y_train_pids_labels.count(1)

# Identify which label belongs to the minority class
if num_copd1_train < num_copd2_train:
    minority_label = 0
    minority_count, majority_count = num_copd1_train, num_copd2_train
else:
    minority_label = 1
    minority_count, majority_count = num_copd2_train, num_copd1_train

# Calculate how many augmentations are needed for the minority class
if minority_count > 0:
    oversampling_factor = majority_count / minority_count
    augmentations_for_minority = round(base_augmentations * oversampling_factor)
else: # Should not happen due to stratify, but good practice
    augmentations_for_minority = base_augmentations

print(f"Training patient distribution: {num_copd1_train} COPD1, {num_copd2_train} COPD2.")
print(f"Each majority class patient will generate {base_augmentations} samples.")
print(f"Each minority class patient will generate {augmentations_for_minority} samples to balance the data.")
# --- OVERSAMPLING LOGIC END ---

X_train, y_train, X_test, y_test = [], [], [], []
for pid in patient_ids_1_2:
    label = label_dict_1_2[pid]
    for side in ['L', 'R']:
        for i in range(1, 7):
            fpath = os.path.join(AUDIO_DIR_1_2, f"{pid}_{side}{i}.wav")
            if not os.path.exists(fpath): continue
            try:
                y_audio, sr = librosa.load(fpath, sr=None)
                if pid in train_pids:
                    num_augmentations = augmentations_for_minority if label == minority_label else base_augmentations
                    y_train.extend([label] * num_augmentations)
                    # Create a list of augmented samples. Add more variations if needed.
                    augs = [
                        extract_log_mel_spectrogram(y_audio, sr),
                        extract_log_mel_spectrogram(add_gaussian_noise(y_audio), sr),
                        extract_log_mel_spectrogram(time_shift(y_audio, sr), sr),
                        extract_log_mel_spectrogram(librosa.effects.pitch_shift(y=y_audio, sr=sr, n_steps=PITCH_SHIFT_STEPS), sr),
                        extract_log_mel_spectrogram(librosa.effects.time_stretch(y=y_audio, rate=1/TIME_STRETCH_RATE), sr),
                        # Add more augmentations to draw from if needed for high oversampling rates
                        extract_log_mel_spectrogram(librosa.effects.pitch_shift(y=y_audio, sr=sr, n_steps=-PITCH_SHIFT_STEPS), sr),
                        extract_log_mel_spectrogram(time_shift(y_audio, sr), sr) # another random time shift
                    ]
                    X_train.extend(augs[:num_augmentations])
                elif pid in test_pids:
                    X_test.append(extract_log_mel_spectrogram(y_audio, sr))
                    y_test.append(label)
            except Exception as e:
                print(f"Warning: Error processing {fpath}: {e}")

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
if len(np.unique(y_train)) < 2: raise ValueError(f"FATAL: Training set has <2 classes due to MISSING audio files in '{AUDIO_DIR_1_2}'.")
X_train, X_test = X_train[..., np.newaxis], X_test[..., np.newaxis]
print("\n--- Final Data Shapes ---\n", f"X_train: {X_train.shape}, y_train: {y_train.shape}\n", f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# --- Step 4: Verify Class Balance ---
print("\n--- Step 4: Verifying Data Balance ---")
print("Since we used oversampling, the training data is now manually balanced.")
print(f"Final training sample counts: COPD1(0)={np.sum(y_train == 0)}, COPD2(1)={np.sum(y_train == 1)}")
print("Class weights are not needed.")


# --- Step 5: Build the CNN Model ---
print("\n--- Step 5: Building the CNN Model for 1-2 Progression ---")
# model_1_2 = Sequential([
#     tf.keras.Input(shape=(N_MELS, MAX_LEN, 1)),
#     Conv2D(32, (3, 3), activation='relu', padding='same'), BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
#     Conv2D(64, (3, 3), activation='relu', padding='same'), BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
#     Conv2D(128, (3, 3), activation='relu', padding='same'), BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)), Dropout(0.3),
#     Flatten(), Dense(128, activation='relu'), Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])
# model_1_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
# model_1_2.summary()

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, Add
from tensorflow.keras.models import Model

def resnet_block(input_tensor, filters):
    """A simple residual block."""
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # This is the "skip connection"
    # It adds the original input back to the output of the convolutional block
    x = Add()([x, input_tensor])
    return x

# --- Define the Model Architecture using the ResNet block ---
input_layer = Input(shape=(N_MELS, MAX_LEN, 1))

# Initial Conv layer to get to the right number of filters (e.g., 64)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)

# --- Add Residual Blocks ---
x = resnet_block(x, filters=64)
x = resnet_block(x, filters=64)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.3)(x)

x = resnet_block(x, filters=64)
x = resnet_block(x, filters=64)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.3)(x)

# --- Classifier Head ---
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

# Create the final model
model_1_2_resnet = Model(inputs=input_layer, outputs=output_layer)
model_1_2_resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model_1_2_resnet.summary()

--- Step 3: Loading Data for COPD 1 vs 2 ---

Performing stratified patient-aware split...

Calculating oversampling rate to balance the training set...
Training patient distribution: 4 COPD1, 5 COPD2.
Each majority class patient will generate 5 samples.
Each minority class patient will generate 6 samples to balance the data.

--- Final Data Shapes ---
 X_train: (588, 128, 150, 1), y_train: (588,)
 X_test: (36, 128, 150, 1), y_test: (36,)

--- Step 4: Verifying Data Balance ---
Since we used oversampling, the training data is now manually balanced.
Final training sample counts: COPD1(0)=288, COPD2(1)=300
Class weights are not needed.

--- Step 5: Building the CNN Model for 1-2 Progression ---


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 150,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 150,  │        640 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 150,  │        256 │ conv2d_15[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 128, 150,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 150,  │        256 │ conv2d_16[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 128, 150,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 150,  │        256 │ conv2d_17[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128, 150,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 128, 150,  │     36,928 │ add_4[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 150,  │        256 │ conv2d_18[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 128, 150,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 150,  │        256 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 128, 150,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 64, 75,    │          0 │ add_5[0][0]       │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 64, 75,    │          0 │ max_pooling2d_8[… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 64, 75,    │     36,928 │ dropout_11[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 75,    │        256 │ conv2d_20[0][0] 

 Total params: 9,997,953 (38.14 MB)

 Trainable params: 9,996,801 (38.13 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [36]:
# --- Step 6: Train the Model ---
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH_1_2, save_best_only=True, monitor='val_accuracy', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
print(f"\n--- Step 6: Starting Model Training on Balanced Data ---\n")
# We DO NOT pass class_weight here, as the data is already balanced
history_1_2 = model_1_2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop, model_checkpoint, reduce_lr])


--- Step 6: Starting Model Training on Balanced Data ---

Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 648ms/step - accuracy: 0.6401 - loss: 0.5192
Epoch 1: val_accuracy improved from -inf to 0.41667, saving model to best_copd_1_2_oversampled_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 669ms/step - accuracy: 0.6412 - loss: 0.5179 - val_accuracy: 0.4167 - val_loss: 0.8251 - learning_rate: 0.0010
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.7133 - loss: 0.4757
Epoch 2: val_accuracy improved from 0.41667 to 0.44444, saving model to best_copd_1_2_oversampled_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 684ms/step - accuracy: 0.7144 - loss: 0.4749 - val_accuracy: 0.4444 - val_loss: 1.0353 - learning_rate: 0.0010
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 667ms/step - accuracy: 0.8180 - loss: 0.3689
Epoch 3: val_accuracy improved from 0.44444 to 0.55556, saving model to best_copd_1_2_oversampled_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 687ms/step - accuracy: 0.8165 - loss: 0.3

In [37]:
# --- Step 7: Evaluate the Final Model ---
print("\n--- Step 7: Evaluating Best Saved Model ---")
model_1_2.load_weights(MODEL_SAVE_PATH_1_2)
loss, accuracy = model_1_2.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy (1 vs 2): {accuracy*100:.2f}%")
print(f"Final Test Loss (1 vs 2): {loss:.4f}")


--- Step 7: Evaluating Best Saved Model ---

Final Test Accuracy (1 vs 2): 55.56%
Final Test Loss (1 vs 2): 0.9790


In [40]:
# --- Step 8: Patient-Level Analysis (Averaging Method with Detailed Scores) ---

def run_patient_analysis_by_average(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    """
    Analyzes patients by averaging scores and also shows the raw prediction
    for each of the 12 audio files per patient.
    """
    print(f"\n--- 📈 Starting Analysis for '{target_diagnosis}' Patients (Averaging Scores) ---")
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    if not patient_ids: return None
    print(f"Found {len(patient_ids)} patients. Analyzing files...")
    results_list = []
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fpath = os.path.join(audio_dir, f"{pid}_{side}{i}.wav")
                if not os.path.exists(fpath): continue
                try:
                    y, sr = librosa.load(fpath, sr=None)
                    spec = extract_log_mel_spectrogram(y, sr)
                    prob = trained_model.predict(np.expand_dims(spec, axis=(0, -1)), verbose=0)[0][0]
                    scores.append(prob)
                except Exception as e:
                    print(f"Warning: Could not process file {fpath}: {e}")

        # If any scores were successfully collected for the patient:
        if scores:
            # --- ADDITION: Print the detailed scores for this patient ---
            formatted_scores = [f'{s:.4f}' for s in scores]
            print(f"  -> Raw Scores for Patient {pid}: {formatted_scores}")
            
            # Append the calculated average to the results list
            results_list.append({'Patient ID': pid, 'Avg_Score': np.mean(scores), 'Audio_Files_Found': len(scores)})
            
    # If no results were generated at all, exit.
    if not results_list:
        print("Could not generate any analysis results. Check if audio files exist.")
        return None

    results_df = pd.DataFrame(results_list).sort_values(by='Avg_Score', ascending=sort_ascending)
    return results_df.reset_index(drop=True)

# Assessment functions remain the same
def assess_copd2_patient(row):
    return "✅ Model Confident (Correctly resembles Stage 2)" if row['Avg_Score'] >= 0.5 else "⚠️ Model Lacks Confidence (False Negative)"

def assess_copd1_progression_risk(row):
    return "⚠️ High Risk (Resembles Stage 2 - False Positive)" if row['Avg_Score'] >= 0.5 else "✅ Low Risk (Correctly identified as Stage 1)"

# --- Run Analysis for COPD2 Patients ---
copd2_avg_df = run_patient_analysis_by_average(model_1_2, LABEL_PATH_1_2, AUDIO_DIR_1_2, 'COPD2', True)
if copd2_avg_df is not None:
    copd2_avg_df['Assessment'] = copd2_avg_df.apply(assess_copd2_patient, axis=1)
    print("\n--- ✅ COPD2 Patient Confidence Results (Averaging Method) ---")
    print("Shows model's confidence in identifying patients known to have Stage 2. Avg_Score >= 0.5 is correct.")
    print(copd2_avg_df.to_string())

# --- Run Analysis for COPD1 Patients ---
copd1_avg_df = run_patient_analysis_by_average(model_1_2, LABEL_PATH_1_2, AUDIO_DIR_1_2, 'COPD1', False)
if copd1_avg_df is not None:
    copd1_avg_df['Assessment'] = copd1_avg_df.apply(assess_copd1_progression_risk, axis=1)
    print("\n--- ⚠️ COPD1 Patient Progression Risk Results (Averaging Method) ---")
    print("Shows which Stage 1 patients are flagged as being at high risk of progressing to Stage 2.")
    print(copd1_avg_df.to_string())


--- 📈 Starting Analysis for 'COPD2' Patients (Averaging Scores) ---
Found 7 patients. Analyzing files...
  -> Raw Scores for Patient H018: ['0.9585', '0.9930', '0.9683', '0.9209', '0.9946', '0.9794', '0.9402', '0.9764', '0.9738', '0.9938', '0.9898', '0.9981']
  -> Raw Scores for Patient H028: ['0.8745', '0.9564', '0.9635', '0.9132', '0.8788', '0.8706', '0.9170', '0.8631', '0.9199', '0.8957', '0.6491', '0.9875']
  -> Raw Scores for Patient H030: ['0.9761', '0.9740', '0.9293', '0.5022', '0.8417', '0.8656', '0.9631', '0.8492', '0.9919', '0.6016', '0.8483', '0.9950']
  -> Raw Scores for Patient H031: ['0.9976', '0.9738', '0.8924', '0.8240', '0.9142', '0.6711', '0.9747', '0.9812', '0.9916', '0.8795', '0.9794', '0.7426']
  -> Raw Scores for Patient H038: ['0.7748', '0.5026', '0.0402', '0.7305', '0.0451', '0.8181', '0.1601', '0.1640', '0.3982', '0.4474', '0.6468', '0.3387']
  -> Raw Scores for Patient H042: ['0.9131', '0.9477', '0.9030', '0.8864', '0.9581', '0.8764', '0.9623', '0.7627', '0.9

In [41]:
# --- Step 8: Patient-Level Analysis (Counting Method with Detailed Scores) ---

def run_patient_analysis_by_vote(trained_model, labels_path, audio_dir, target_diagnosis, sort_ascending):
    """
    Analyzes patients using a majority vote method. It first shows the raw
    prediction for each audio file, then counts the votes to determine the
    final patient-level prediction.
    """
    print(f"\n--- 🗳️ Starting Analysis for '{target_diagnosis}' Patients (Majority Vote) ---")
    df_labels = pd.read_excel(labels_path)
    patient_ids = df_labels[df_labels['Diagnosis'] == target_diagnosis]['Patient ID'].tolist()
    if not patient_ids: return None
    print(f"Found {len(patient_ids)} patients. Analyzing files and counting votes...")
    
    patient_scores_data = {}
    for pid in patient_ids:
        scores = []
        for side in ['L', 'R']:
            for i in range(1, 7):
                fpath = os.path.join(audio_dir, f"{pid}_{side}{i}.wav")
                if not os.path.exists(fpath): continue
                try:
                    y, sr = librosa.load(fpath, sr=None)
                    spec = extract_log_mel_spectrogram(y, sr)
                    prob = trained_model.predict(np.expand_dims(spec, axis=(0, -1)), verbose=0)[0][0]
                    scores.append(prob)
                except Exception as e:
                    print(f"Warning: Could not process file {fpath}: {e}")
        
        # If scores were collected, show the details and store them
        if scores:
            # --- ADDITION: Print the detailed scores for this patient ---
            formatted_scores = [f'{s:.4f}' for s in scores]
            print(f"  -> Raw Scores for Patient {pid}: {formatted_scores}")
            
            patient_scores_data[pid] = scores

    # If no patient data could be processed, exit.
    if not patient_scores_data:
        print("Could not generate any analysis results. Check if audio files exist.")
        return None
        
    results_list = []
    for pid, scores in patient_scores_data.items():
        # --- COUNTING LOGIC ---
        copd2_votes = sum(1 for s in scores if s >= 0.5)
        copd1_votes = len(scores) - copd2_votes
        
        # Determine final prediction by majority (defaulting to Stage 1 on a tie)
        final_prediction = 'COPD2' if copd2_votes > copd1_votes else 'COPD1'
            
        results_list.append({
            'Patient ID': pid, 'COPD2_Votes': copd2_votes, 'COPD1_Votes': copd1_votes,
            'Total_Files': len(scores), 'Final_Prediction': final_prediction
        })

    results_df = pd.DataFrame(results_list).sort_values(by='COPD2_Votes', ascending=sort_ascending)
    return results_df.reset_index(drop=True)

def assess_prediction_vs_truth(row, true_label):
    """Compares the majority vote prediction to the known truth."""
    if row['Final_Prediction'] == true_label:
        return f"✅ Correct (Predicted {row['Final_Prediction']})"
    else:
        return f"❌ Incorrect (Predicted {row['Final_Prediction']}, but was {true_label})"


# --- Run Analysis on COPD2 Patients (High-Severity Group) ---
copd2_vote_df = run_patient_analysis_by_vote(
    trained_model=model_1_2, labels_path=LABEL_PATH_1_2, audio_dir=AUDIO_DIR_1_2,
    target_diagnosis='COPD2', sort_ascending=True # Show patients with FEWEST votes for COPD2 first
)
if copd2_vote_df is not None:
    copd2_vote_df['Assessment'] = copd2_vote_df.apply(assess_prediction_vs_truth, true_label='COPD2', axis=1)
    print("\n--- ✅ COPD2 Patient Majority Vote Results ---")
    print("This table shows if the model's majority vote matched the patient's actual 'COPD2' diagnosis.")
    print(copd2_vote_df.to_string())

# --- Run Analysis on COPD1 Patients (Progression Risk Group) ---
copd1_vote_df = run_patient_analysis_by_vote(
    trained_model=model_1_2, labels_path=LABEL_PATH_1_2, audio_dir=AUDIO_DIR_1_2,
    target_diagnosis='COPD1', sort_ascending=False # Show Stage 1 patients with MOST votes for COPD2 first
)
if copd1_vote_df is not None:
    copd1_vote_df['Assessment'] = copd1_vote_df.apply(assess_prediction_vs_truth, true_label='COPD1', axis=1)
    print("\n--- ⚠️ COPD1 Patient Progression Risk Results ---")
    print("This table shows if a Stage 1 patient is incorrectly flagged as having progressed to Stage 2.")
    print("Incorrect '❌' assessments here are patients the model thinks are at HIGH RISK of progression.")
    print(copd1_vote_df.to_string())


--- 🗳️ Starting Analysis for 'COPD2' Patients (Majority Vote) ---
Found 7 patients. Analyzing files and counting votes...
  -> Raw Scores for Patient H018: ['0.9585', '0.9930', '0.9683', '0.9209', '0.9946', '0.9794', '0.9402', '0.9764', '0.9738', '0.9938', '0.9898', '0.9981']
  -> Raw Scores for Patient H028: ['0.8745', '0.9564', '0.9635', '0.9132', '0.8788', '0.8706', '0.9170', '0.8631', '0.9199', '0.8957', '0.6491', '0.9875']
  -> Raw Scores for Patient H030: ['0.9761', '0.9740', '0.9293', '0.5022', '0.8417', '0.8656', '0.9631', '0.8492', '0.9919', '0.6016', '0.8483', '0.9950']
  -> Raw Scores for Patient H031: ['0.9976', '0.9738', '0.8924', '0.8240', '0.9142', '0.6711', '0.9747', '0.9812', '0.9916', '0.8795', '0.9794', '0.7426']
  -> Raw Scores for Patient H038: ['0.7748', '0.5026', '0.0402', '0.7305', '0.0451', '0.8181', '0.1601', '0.1640', '0.3982', '0.4474', '0.6468', '0.3387']
  -> Raw Scores for Patient H042: ['0.9131', '0.9477', '0.9030', '0.8864', '0.9581', '0.8764', '0.9623